In [40]:
# Import dependenc
import pandas as pd
from pymongo import MongoClient
from pprint import pprint

In [41]:
# Import Mongo keys
from mongo_access  import password

In [42]:
# Create an instance of MongoClient
mongo = MongoClient(password)

In [43]:
# Assign the database to a variable name
db = mongo['books']

In [44]:
# review the collections in our new database
print(db.list_collection_names())

['books']


In [45]:
# assign the collection to a variable
collection = db['books']

In [46]:
# Create a query that finds the books that were published before 1999
before_1999 = {'publication_year': {'$lt': "1999"}}

# Capture the results to a variable
results = collection.find(before_1999)

# Pretty print the first 10 results
for i in range(10):
    pprint(results[0])

{'_id': ObjectId('66da49047084538b3e00f9c2'),
 'asin': '',
 'authors': [{'author_id': '604031', 'role': ''}],
 'average_rating': '4.00',
 'book_id': '5333265',
 'country_code': 'US',
 'description': '',
 'edition_information': '',
 'format': 'Paperback',
 'image_url': 'https://images.gr-assets.com/books/1310220028m/5333265.jpg',
 'is_ebook': 'false',
 'isbn': '0312853122',
 'isbn13': '9780312853129',
 'kindle_asin': '',
 'language_code': '',
 'link': 'https://www.goodreads.com/book/show/5333265-w-c-fields',
 'num_pages': '256',
 'popular_shelves': [{'count': '3', 'name': 'to-read'},
                     {'count': '1', 'name': 'p'},
                     {'count': '1', 'name': 'collection'},
                     {'count': '1', 'name': 'w-c-fields'},
                     {'count': '1', 'name': 'biography'}],
 'publication_day': '1',
 'publication_month': '9',
 'publication_year': '1984',
 'publisher': "St. Martin's Press",
 'ratings_count': '3',
 'series': [],
 'similar_books': [],
 'text

In [47]:
# Create a query that finds the books that were published before in the last 10 years
after_2007 = {'publication_year': {'$gte': "2007"}}

# Capture the results to a variable
results = collection.find(after_2007)

# Pretty print the first 10 results
for i in range(10):
    pprint(results[0])

{'_id': ObjectId('66da49047084538b3e00f9c5'),
 'asin': '',
 'authors': [{'author_id': '9212', 'role': ''}],
 'average_rating': '3.49',
 'book_id': '6066819',
 'country_code': 'US',
 'description': 'Addie Downs and Valerie Adler were eight when they first met '
                'and decided to be best friends forever. But, in the wake of '
                'tragedy and betrayal during their teenage years, everything '
                'changed. Val went on to fame and fortune. Addie stayed behind '
                'in their small Midwestern town. Destiny, however, had more in '
                'store for these two. And when, twenty-five years later, Val '
                "shows up at Addie's front door with blood on her coat and "
                'terror on her face, it is the beginning of a wild adventure '
                'for two women joined by love and history who find strength '
                'together that they could not find alone.',
 'edition_information': '',
 'format': 'Hardco

In [48]:
# Find the top 10 books shelved as "to-read"
pipeline = [
     {"$unwind": "$popular_shelves"},
    # Match documents where the shelf name is "to-read"
    {"$match": {"popular_shelves.name": "to-read"}},
    # Group by book ID and sum the count
    {"$group": {
        "_id": "$_id",
        "title": {"$first": "$title"},
        "to_read_count": {"$sum": {"$toInt": "$popular_shelves.count"}}
    }},
    # Sort by to_read_count in descending order
    {"$sort": {"to_read_count": -1}},
    # Limit to top 10 results
    {"$limit": 10}
] 
results = list(collection.aggregate(pipeline))
print("Top 10 books shelved as 'to-read':")
for i, book in enumerate(results, 1):
    print(f"{i}. '{book['title']}'- {book['to_read_count']} times")

Top 10 books shelved as 'to-read':
1. 'The Book Thief'- 835383 times
2. 'Hoțul de cărți'- 835383 times
3. 'The Book Thief'- 835383 times
4. 'The Book Thief'- 835383 times
5. 'La Voleuse de livres'- 835383 times
6. 'Boktyven'- 835383 times
7. 'Boktjuven'- 835383 times
8. 'The Book Thief'- 835383 times
9. 'A menina que roubava livros'- 835383 times
10. 'La Voleuse de livres'- 835383 times


In [27]:
# Find the top 10 books with the most written reviews
pipeline = [
    {"$addFields": {
        "text_reviews_count_int": {
            "$cond": {
                "if": {"$and": [
                    {"$ne": ["$text_reviews_count", ""]},
                    {"$ne": ["$text_reviews_count", None]},
            
                ]},
                "then": {"$toInt": "$text_reviews_count"},
                "else": None
            }
        }
    }},
    {"$match": {"text_reviews_count_int": {"$ne": None}}},
    {"$group": {"_id": "$text_reviews_count_int", "count": {"$sum": 1}}},
    {"$sort": {"_id", -1}},
    {"$limit": 10}
]

result = list(collection.aggregate(pipeline))

# Print the results
#for item in result:
    print(f"Text Reviews Count: {book['title']}, Occurrences: {item['count']}")


InvalidDocument: cannot encode object: {'_id', -1}, of type: <class 'set'>

In [49]:
# Find how many books have the language as "Spanish"
spanish_books = collection.count_documents({'language_code': 'spa'})
spanish_books

55509

In [52]:
# Write an aggregation query that counts the number of books, grouped by "format"


In [38]:
# Create a query that finds the top 10 books that have an average rating of more that 4.0
query = {'average_rating': {'$gte': "4.0"}}

# Capture the results to a variable
results = collection.find(query)

# Pretty print the first 10 results
for i in range(10):
    pprint(results[0])

{'_id': ObjectId('66da49047084538b3e00f9c2'),
 'asin': '',
 'authors': [{'author_id': '604031', 'role': ''}],
 'average_rating': '4.00',
 'book_id': '5333265',
 'country_code': 'US',
 'description': '',
 'edition_information': '',
 'format': 'Paperback',
 'image_url': 'https://images.gr-assets.com/books/1310220028m/5333265.jpg',
 'is_ebook': 'false',
 'isbn': '0312853122',
 'isbn13': '9780312853129',
 'kindle_asin': '',
 'language_code': '',
 'link': 'https://www.goodreads.com/book/show/5333265-w-c-fields',
 'num_pages': '256',
 'popular_shelves': [{'count': '3', 'name': 'to-read'},
                     {'count': '1', 'name': 'p'},
                     {'count': '1', 'name': 'collection'},
                     {'count': '1', 'name': 'w-c-fields'},
                     {'count': '1', 'name': 'biography'}],
 'publication_day': '1',
 'publication_month': '9',
 'publication_year': '1984',
 'publisher': "St. Martin's Press",
 'ratings_count': '3',
 'series': [],
 'similar_books': [],
 'text

In [39]:
# Create a query of books that have at least 300 ratings and scored more than 3.5 
# results = 

SyntaxError: invalid syntax (2524325341.py, line 2)

In [ ]:
# Convert mongo result to Pandas DataFrame
result_df = pd.DataFrame(results)

print("Rows in DataFrame: ", len(result_df))
result_df.head(10)